### This notebook is for Coursera Capstone assignment

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

### Get Wikipedia HTML page for List of postal codes of Canada: M

In [ ]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

### Find post code table within the HTML file

In [ ]:
zip_table = soup.findAll('table', {'class':'wikitable'})[0].tbody

In [ ]:
zip_rows = zip_table.find_all('tr')
#zip_rows

In [ ]:
all_post_codes = []
for tag in zip_rows:
    row = tag.text
    post_code = (row.split('\n')[1:4])
    all_post_codes.append(post_code)

#Remove the first row, as it is the title of the table
all_post_codes = all_post_codes[1:]

In [ ]:
len(all_post_codes)

In [ ]:
col_names = ['Postcode', 'Borough', 'Neighbourhood']
zip_df = pd.DataFrame(all_post_codes,columns=col_names)


In [ ]:
zip_df.head(10)

### Exclude post code rows, which dont have a Borough assigned

In [ ]:
zip_cleaned_df = zip_df.loc[zip_df['Borough'] != 'Not assigned'].reset_index(drop=True)
zip_cleaned_df.shape
zip_cleaned_df.head()

### Combined the neighborhoods, with same post code

In [ ]:
neighborhood_groups = zip_cleaned_df.groupby('Postcode').groups
duplicates_to_remove = []
for key, post_indexes in neighborhood_groups.items():
    
    if(post_indexes.size > 1):
        #print(key, post_index)
        
        #Iterate over the groups and get Neighborhoods
        neighbourhoods = ''
        for post_index in post_indexes:
            #print(post_index)
            #print(key, zip_cleaned_df.iloc[post_index].Neighbourhood)
            #Combine all the neighbourhoods
            neighbourhoods += (zip_cleaned_df.iloc[post_index].Neighbourhood + ', ')
        
        #trim the last ',' and the space
        neighbourhoods = neighbourhoods[:-2]
        
        first_index = post_indexes[0]
        #print(key, neighbourhoods)
        #Assign the combined string into first index in the dataframe
        zip_cleaned_df.at[first_index, 'Neighbourhood'] = neighbourhoods
        #print(post_indexes[1:].tolist())
        duplicates_to_remove.extend(post_indexes[1:].tolist())

In [ ]:
#Remove duplicate Boroughs from dataframe
print(duplicates_to_remove)
zip_cleaned_df.drop(duplicates_to_remove, inplace=True)
#print()
    
    

In [ ]:
zip_cleaned_df.shape
zip_cleaned_df.head(10)

In [ ]:
zip_cleaned_df.loc[(zip_cleaned_df['Borough'] != 'Not assigned') & (zip_cleaned_df['Neighbourhood'] == 'Not assigned')]

### Replace values in Neighbourhood column with Borough, if it is 'Not assigned'

In [ ]:
# for row in zip_cleaned_df.itertuples():
    
#     if row.Neighbourhood == 'Not assigned':
        
#         print(row)
#         borough = row.Borough
#         ind_to_update = row.Index
#         zip_cleaned_df[ind_to_update, 'zip_cleaned_df'] = borough
#         break;
zip_cleaned_df.at[6, 'Neighbourhood'] = "Queen's Park"

In [ ]:
zip_cleaned_df.shape